In [1]:
import lionagi as li

In [2]:
from pathlib import Path

data_path = Path.cwd() / "lionagi_data"  # Path to the data directory

### Prepare Data

In [3]:
# load
docs = li.load(input_dir=data_path, recursive=True, required_exts=[".py"])
docs = [doc for doc in docs if len(doc.content) > 50]

# chunk
pile = li.chunk(docs=docs, chunk_size=2000, overlap=0.1)

In [4]:
from llama_index.core.postprocessor import LLMRerank

pile.setup_query_engine(
    index_type = "llama_index",
    node_postprocessor = LLMRerank()
)

await pile.query_pile("what is lionagi?")

'lionagi is a Python package that consists of various modules and functionalities such as system utilities, asynchronous utilities, data manipulation tools, API utilities, validation functions, and integration services like LiteLLMService.'

### Construct Workflow

In [5]:
instruction = """
write a good API documentation for this code, must use 
query engine to check meanings of related code concepts 
to accurately describe, for example if a name of a variable,
function, class, or module is used but not present in context,
you must check with the query engine. make sure to cross 
reference the code with the query engine to ensure the 
documentation is accurate
"""

edit = """
provide documentation only: final documentation in md 
format of the module of interest, do not include other 
fields do not present in JSON format, only markdown format 
you asked a lot of good questions and got plenty answers, 
please integrate your conversation, be a lot more technical, you will 
be rewarded with 500 dollars for great work, and 
punished for subpar work, take a deep breath, you can do it
"""

In [6]:
from PROMPTS import sys_prompt  # put your system prompt here

model = li.iModel(
    model="gpt-4o",
    provider="openai",
    interval_tokens=1_000_000,
    interval_requests=1_000,
    interval=60,
)

tools = pile.to_query_tool()

async def write_doc(context):
    branch = li.Branch(
        system=sys_prompt, 
        tools=tools, 
        imodel=model
    )

    form = await branch.direct(
        instruction=instruction,
        context=context,
        reason=True,
        score=True,
        allow_action=True,
        allow_extension=True,
        max_extension=3,
    )
    
    if form is None:
        print("No form returned")
        return None, None

    form.answer = await branch.chat(
        instruction=edit,
        temperature=0.5,
    )
    
    # save all messages into a unique file
    df = branch.to_df()
    df.to_csv(f"lion_doc_{branch.ln_id[:8]}.csv", index=False)

    return form, branch


contexts = [i.content for i in docs]

### Run workflow

In [7]:
form, branch = await write_doc(contexts[82])

In [8]:
form.display()

**task**: 
 Follow the prompt and provide the necessary output.
- Additional instruction: 
write a good API documentation for this code, must use 
query engine to check meanings of related code concepts 
to accurately describe, for example if a name of a variable,
function, class, or module is used but not present in context,
you must check with the query engine. make sure to cross 
reference the code with the query engine to ensure the 
documentation is accurate

- Additional context: """
Copyright 2024 HaiyangLi
Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at
 http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
"""
from collections.abc import Mapping
from collections import deque
from typing import Tuple
from pydantic import Field
import contextlib
from .abc import (
 Record,
 LionTypeError,
 ItemNotFoundError,
 LionIDable,
 Element,
)
from .pile import Pile, pile
from .progression import Progression, progression

class Flow(Element):
 """
 Represents a flow of categorical sequences.
 Attributes:
  sequences (Pile[Progression]): A collection of progression sequences.
  registry (dict[str, str]): A registry mapping sequence names to IDs.
  default_name (str): The default name for the flow.
 """
 sequences: Pile[Progression] = Field(
  default_factory=lambda: pile({}, Progression, use_obj=True)
 )
 registry: dict[str, str] = {}
 default_name: str = "main"
 def __init__(self, sequences=None, default_name=None):
  """
  Initializes a Flow instance.
  Args:
   sequences (optional): Initial sequences to include in the flow.
   default_name (optional): Default name for the...

**reason**: 
 Let's think step by step. To write accurate API documentation, we need to understand the meaning of certain classes, functions, and variables used in the code. This requires querying the meanings of related code concepts that are not present in the context.

**actions**: 
 
 
1. **query**(query: Pile class in Python), 
 
2. **query**(query: Progression class in Python), 
 
3. **query**(query: LionTypeError class in Python), 
 
4. **query**(query: ItemNotFoundError class in Python), 
 
5. **query**(query: LionIDable class in Python), 
 
6. **query**(query: Element class in Python)

**action_required**: True

**extension_required**: True

**score**: 8

**action_response**: 
 
 
1. **query**(query: Pile class in Python) 
 
 The Pile class in Python is a versatile container for managing collections of Element objects. It supports structured access and manipulation, including retrieval, addition, and deletion of elements. The class facilitates ordered and type-validated storage and access, supporting both index-based and key-based retrieval. It includes attributes such as 'pile' for mapping unique identifiers to items, 'item_type' for allowed item types, 'name' for an optional name for the pile, 'order' for the order of item identifiers, and 'use_obj' to determine if Record and Ordering should be treated as objects., 
 
2. **query**(query: Progression class in Python) 
 
 The Progression class in Python defines a data structure that represents a progression of items. It includes methods to add, remove, iterate over, clear, and manipulate items within the progression. The class provides functionality to extend the progression, include items, remove items, return the length of the progression, iterate over items, clear the progression, convert the progression to a dictionary, and more., 
 
3. **query**(query: LionTypeError class in Python) 
 
 The LionTypeError class in Python is designed to handle exceptions raised for type mismatch or type checking errors., 
 
4. **query**(query: ItemNotFoundError class in Python) 
 
 The ItemNotFoundError class in Python is designed to be raised when a specified item is not found., 
 
5. **query**(query: LionIDable class in Python) 
 
 The LionIDable class in Python is part of the LionAGI abstract base classes. It is used for managing a collection of unique LionAGI items and is designed to accept LionIDable for retrieval and require Component instances for addition. The LionIDable class provides methods for retrieving items by identifier and interacting with the collection of items in the record., 
 
6. **query**(query: Element class in Python) 
 
 The Element class in Python is a base class that is used in the context to manage collections of objects. It supports structured access and manipulation of elements within a container, allowing for retrieval, addition, and deletion of items.

**action_performed**: True

**available_tools**: ['query']

**answer**: 
 # Flow Class API Documentation

## Class: `Flow`

**Description**:
`Flow` represents a flow of categorical sequences. It manages a collection of progression sequences and provides various methods to manipulate and access these sequences.

**Attributes**:
- `sequences` (`Pile[Progression]`): A collection of progression sequences.
- `registry` (`dict[str, str]`): A registry mapping sequence names to IDs.
- `default_name` (`str`): The default name for the flow.

### Method: `__init__`

**Signature**:
```python
def __init__(self, sequences=None, default_name=None):
```

**Parameters**:
- `sequences` (optional): Initial sequences to include in the flow.
- `default_name` (optional): Default name for the flow.

**Return Values**:
- None

### Method: `_validate_sequences`

**Signature**:
```python
def _validate_sequences(self, value):
```

**Parameters**:
- `value`: Sequences to validate and initialize.

**Return Values**:
- `Pile[Progression]`: A pile of progression sequences.

### Method: `all_orders`

**Signature**:
```python
def all_orders(self) -> list[list[str]]:
```

**Parameters**:
- None

**Return Values**:
- `list[list[str]]`: A list of lists containing sequence orders.

### Method: `all_unique_items`

**Signature**:
```python
def all_unique_items(self) -> Tuple[str]:
```

**Parameters**:
- None

**Return Values**:
- `Tuple[str]`: A tuple of unique items.

### Method: `keys`

**Signature**:
```python
def keys(self):
```

**Parameters**:
- None

**Return Values**:
- Yields keys from the sequences.

### Method: `values`

**Signature**:
```python
def values(self):
```

**Parameters**:
- None

**Return Values**:
- Yields values from the sequences.

### Method: `items`

**Signature**:
```python
def items(self):
```

**Parameters**:
- None

**Return Values**:
- Yields items from the sequences.

### Method: `get`

**Signature**:
```python
def get(self, seq=None, default=...):
```

**Parameters**:
- `seq` (optional): The name of the sequence.
- `default` (optional): Default value if sequence is not found.

**Return Values**:
- `Progression`: The requested sequence.

### Method: `__getitem__`

**Signature**:
```python
def __getitem__(self, seq=None, /):
```

**Parameters**:
- `seq` (optional): The name of the sequence.

**Return Values**:
- The requested sequence.

### Method: `__setitem__`

**Signature**:
```python
def __setitem__(self, seq: LionIDable | str, index=None, value=None, /):
```

**Parameters**:
- `seq` (`LionIDable | str`): The sequence identifier.
- `index` (optional): The index within the sequence.
- `value` (optional): The value to set.

**Return Values**:
- None

### Method: `__contains__`

**Signature**:
```python
def __contains__(self, item):
```

**Parameters**:
- `item`: The item to check for.

**Return Values**:
- `bool`: True if the item is in the flow, False otherwise.

### Method: `shape`

**Signature**:
```python
def shape(self):
```

**Parameters**:
- None

**Return Values**:
- The shape of the sequences.

### Method: `size`

**Signature**:
```python
def size(self):
```

**Parameters**:
- None

**Return Values**:
- The size of the sequences.

### Method: `clear`

**Signature**:
```python
def clear(self):
```

**Parameters**:
- None

**Return Values**:
- None

### Method: `include`

**Signature**:
```python
def include(self, seq=None, item=None, name=None):
```

**Parameters**:
- `seq` (optional): The sequence to include.
- `item` (optional): The item to include.
- `name` (optional): The name of the sequence.

**Return Values**:
- `bool`: True if inclusion was successful, False otherwise.

### Method: `exclude`

**Signature**:
```python
def exclude(self, seq: LionIDable = None, item=None, name=None):
```

**Parameters**:
- `seq` (`LionIDable`, optional): The sequence to exclude from.
- `item` (optional): The item to exclude.
- `name` (optional): The name of the sequence.

**Return Values**:
- `bool`: True if exclusion was successful, False otherwise.

### Method: `register`

**Signature**:
```python
def register(self, sequence: Progression, name: str = None):
```

**Parameters**:
- `sequence` (`Progression`): The sequence to register.
- `name` (`str`, optional): The name for the sequence.

**Return Values**:
- None

**Exceptions Raised**:
- `LionTypeError`: If the sequence is not of type Progression.
- `ValueError`: If the sequence name already exists.

### Method: `append`

**Signature**:
```python
def append(self, item, sequence=None, /):
```

**Parameters**:
- `item`: The item to append.
- `sequence` (optional): The sequence to append to.

**Return Values**:
- None

### Method: `popleft`

**Signature**:
```python
def popleft(self, sequence=None, /):
```

**Parameters**:
- `sequence` (optional): The sequence to remove the item from.

**Return Values**:
- The removed item.

### Method: `remove`

**Signature**:
```python
def remove(self, item, sequence="all"):
```

**Parameters**:
- `item`: The item to remove.
- `sequence` (`str`, optional): The sequence to remove the item from. Defaults to "all".

**Return Values**:
- None

### Method: `__len__`

**Signature**:
```python
def __len__(self):
```

**Parameters**:
- None

**Return Values**:
- The length of the sequences.

### Method: `__iter__`

**Signature**:
```python
def __iter__(self):
```

**Parameters**:
- None

**Return Values**:
- An iterator over the sequences.

### Method: `__next__`

**Signature**:
```python
def __next__(self):
```

**Parameters**:
- None

**Return Values**:
- The next item in the sequences.

### Method: `_find_sequence`

**Signature**:
```python
def _find_sequence(self, sequence=None, default=...):
```

**Parameters**:
- `sequence` (optional): The sequence to find.
- `default` (optional): The default value if sequence is not found.

**Return Values**:
- The found sequence ID.

**Exceptions Raised**:
- `ItemNotFoundError`: If no sequence is found.

### Method: `to_df`

**Signature**:
```python
def to_df(self):
```

**Parameters**:
- None

**Return Values**:
- The sequences as a DataFrame.

## Function: `flow`

**Signature**:
```python
def flow(sequences=None, default_name=None, /):
```

**Parameters**:
- `sequences` (optional): Initial sequences to include in the flow.
- `default_name` (optional): Default name for the flow.

**Return Values**:
- `Flow`: A new Flow instance.

**Usage Examples**:
```python
# Example 1: Creating a new Flow instance with default parameters
flow_instance = flow()

# Example 2: Creating a new Flow instance with initial sequences
sequences = {"seq1": ["item1", "item2"], "seq2": ["item3", "item4"]}
flow_instance = flow(sequences=sequences)

# Example 3: Creating a new Flow instance with a default name
flow_instance = flow(default_name="custom_flow")
```

In [9]:
branch.to_df()

,ln_id,message_type,timestamp,role,content,metadata,sender,recipient
0,5dc8736c19fa35215dca88fd27ebc7a3,System,2024-05-23T12:21:22.292319,system,"{'system_info': ' you are a helpful assistant,...",{'last_updated': {'recipient': '2024-05-23T12:...,system,a36ee8b645040eaabd0a735ebe5ba980
1,ff5d275d31aca540e787b1bb917c87f5,Instruction,2024-05-23T12:21:22.293349,user,{'instruction': ' ## Task Instructions...,{'last_updated': {'sender': '2024-05-23T12:21:...,user,a36ee8b645040eaabd0a735ebe5ba980
2,927acdd1fff0976865d4d764f9849681,AssistantResponse,2024-05-23T12:21:29.831116,assistant,"{'assistant_response': '```json { ""answer""...",{'last_updated': {'sender': '2024-05-23T12:21:...,a36ee8b645040eaabd0a735ebe5ba980,user
3,6c249132bd2eaecc789ce9501042595b,ActionRequest,2024-05-23T12:21:29.832478,assistant,"{'action_request': {'function': 'query', 'argu...",{'last_updated': {'function': '2024-05-23T12:2...,a36ee8b645040eaabd0a735ebe5ba980,febafd4b95d45a24bbd7a867bb088cd0
4,62025b06857ea5c198681eabea7eeec3,ActionRequest,2024-05-23T12:21:29.832618,assistant,"{'action_request': {'function': 'query', 'argu...",{'last_updated': {'function': '2024-05-23T12:2...,a36ee8b645040eaabd0a735ebe5ba980,febafd4b95d45a24bbd7a867bb088cd0
5,955ae1b01bc089a62ef8f53af614c99e,ActionRequest,2024-05-23T12:21:29.832692,assistant,"{'action_request': {'function': 'query', 'argu...",{'last_updated': {'function': '2024-05-23T12:2...,a36ee8b645040eaabd0a735ebe5ba980,febafd4b95d45a24bbd7a867bb088cd0
6,376621b00537ddd6e34a8f86157827a7,ActionRequest,2024-05-23T12:21:29.832755,assistant,"{'action_request': {'function': 'query', 'argu...",{'last_updated': {'function': '2024-05-23T12:2...,a36ee8b645040eaabd0a735ebe5ba980,febafd4b95d45a24bbd7a867bb088cd0
7,8488ec11124d654c90f901ffa61be5dd,ActionRequest,2024-05-23T12:21:29.832815,assistant,"{'action_request': {'function': 'query', 'argu...",{'last_updated': {'function': '2024-05-23T12:2...,a36ee8b645040eaabd0a735ebe5ba980,febafd4b95d45a24bbd7a867bb088cd0
8,883bff02dca5ccfe57a8edff22e14f03,ActionRequest,2024-05-23T12:21:29.832875,assistant,"{'action_request': {'function': 'query', 'argu...",{'last_updated': {'function': '2024-05-23T12:2...,a36ee8b645040eaabd0a735ebe5ba980,febafd4b95d45a24bbd7a867bb088cd0
9,c356005e650c3a2a4084904461c53013,ActionResponse,2024-05-23T12:21:33.177098,assistant,"{'action_response': {'function': 'query', 'arg...",{'last_updated': {'function': '2024-05-23T12:2...,febafd4b95d45a24bbd7a867bb088cd0,a36ee8b645040eaabd0a735ebe5ba980


In [10]:
msg = branch.messages[2]

In [11]:
form.tool_schema

{'tools': [{'type': 'function',
   'function': {'name': 'query',
    'description': 'No description available.',
    'parameters': {'type': 'object',
     'properties': {'query': {'type': 'string',
       'description': 'No description available.'}},
     'required': ['query']}}}]}

In [12]:
form

ln_id                                  9e67a38502c7117a41c5366e4bb57652
created                                      2024-05-23T12:21:22.292993
metadata              {'last_updated': {'input_fields': '2024-05-23T...
content                                                            None
template_name                                             UnitDirective
assignment                                               task -> answer
input_fields                                        [task, tool_schema]
requested_fields      [answer, reason, actions, action_required, ext...
task                  Follow the prompt and provide the necessary ou...
validation_kwargs     {'reason': {}, 'actions': {}, 'action_required...
confidence_score                                                   None
reason                Let's think step by step. To write accurate AP...
actions               {'action_1': {'function': 'query', 'arguments'...
action_required                                                 

In [13]:
if hasattr(form, "extension_forms"):
    for i in form.extension_forms:
        i.display()

**task**: 
 Follow the prompt and provide the necessary output.
- Additional instruction: N/A
- Additional context: N/A
- Reason and prepare actions with GIVEN TOOLS ONLY.
- Allow auto-extension up to another 2 rounds.
- Give a numeric score in [0, 10] and precision of 0 decimal places.


**reason**: 
 Let's think step by step. We have gathered the necessary information about the classes and functions used in the code. Now, we can proceed to write the API documentation accurately.

**actions**: 

**action_required**: False

**extension_required**: False

**score**: 9

**available_tools**: ['query']

**answer**: 
 The API documentation for the provided code is as follows: